In [1]:
import gradio as gr
import pickle

/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(f"./model/carrer_growth_matters_model.pkl", "rb") as f:
    carrer_growth_matters_model = pickle.load(f)
with open(f"./model/company_matters_model.pkl", "rb") as g:
    company_matters_model = pickle.load(g)
with open(f"./model/compensation_matters_model.pkl", "rb") as h:
    compensation_matters_model = pickle.load(h)
with open(f"./model/scaler.pkl", "rb") as j:
    scaler = pickle.load(j)


In [3]:
def get_stock_mapping(stock):
    stock_mapping = {
        "None" : 0,
        "Low" : 1,
        "Medium" : 2,
        "High" : 3,
    }
    return stock_mapping.get(stock, -1)

def get_environment_satisfaction_mapping(environment):
    environment_satisfaction_mapping = {
        "Low": 0,
        "Medium": 1,
        "High": 2,
        "Very High": 3
    }
    return environment_satisfaction_mapping.get(environment, -1)

def get_job_satisfaction_mapping(job):
    job_satisfaction_mapping = {
        "Low": 0,
        "Medium": 1,
        "High": 2,
        "Very High": 3
    }
    return job_satisfaction_mapping.get(job, -1)

def get_over_time_mapping(over_time):
    over_time_mapping = {
        "No": 0,
        "Yes": 1
    }
    return over_time_mapping.get(over_time, -1)

def get_relation_satisfaction_mapping(relation):
    relation_satisfaction_mapping = {
        "Low": 0,
        "Medium": 1,
        "High": 2,
        "Very High": 3
    }
    return relation_satisfaction_mapping.get(relation, -1)

def predict_attrition(_compensation, _company, _career_growth, 
        _monthly_income, _stock_option_level,
        _environment_satisfaction, 
        _job_satisfaction, _over_time, _relationship_satisfaction, 
        _total_working_years, years_in_current_role,
        _years_since_last_promotion):
    scaled_data = scaler.transform([[0, 0, 0, 
                                    0, _monthly_income, 0, 
                                    0, 0, _total_working_years, 
                                    0, 0, years_in_current_role, 
                                    _years_since_last_promotion, 0]])
    weights = {"compensation": 0.375,
                "career_growth": 0.325,
                "company": 0.30,
                }
    votes = {}
    checked = {"compensation": 0,
                "career_growth": 0,
                "company": 0,
                }
    
    if _compensation:
        checked["compensation"] = 1
        monthly_income = scaled_data[0][4]
        stock_option_level = get_stock_mapping(_stock_option_level)
        compensation_prediction = compensation_matters_model.predict([[monthly_income, stock_option_level]])[0]
        if compensation_prediction == 1:
            votes["compensation"] = 1
    if _company:
        checked["company"] = 1
        environment_satisfaction = get_environment_satisfaction_mapping(_environment_satisfaction)
        job_satisfaction = get_job_satisfaction_mapping(_job_satisfaction)
        over_time = get_over_time_mapping(_over_time)
        relationship_satisfaction = get_relation_satisfaction_mapping(_relationship_satisfaction)
        company_prediction = company_matters_model.predict([[environment_satisfaction, job_satisfaction, over_time, relationship_satisfaction]])[0]
        if company_prediction == 1:
            votes["company"] = 1
    if _career_growth:
        checked["career_growth"] = 1
        total_working_years = scaled_data[0][8]
        years_in_current_role = scaled_data[0][11]
        years_since_last_promotion = scaled_data[0][12]
        career_growth_prediction = carrer_growth_matters_model.predict([[years_since_last_promotion, total_working_years, years_in_current_role]])[0]
        if career_growth_prediction == 1:
            votes["career_growth"] = 1
    
    if sum(checked.values()) == 0:
        return "Please check at least one checkbox to get the prediction!"
    
    # if sum(checked.values()) % 2 == 0:
    attrition_score = sum(weights[model] for model, vote in votes.items() if vote == 1)
    no_attrition_score = sum(weights[model] for model, vote in votes.items() if vote == 0)
    return "The Employee might ATTRITION" if attrition_score > no_attrition_score else "Employee should STAY"
    # else:
    #     is_attrition_true = len([value for key, value in votes.items() if value == 1])
    #     is_attrition_false = len([value for key, value in votes.items() if value == 0])
    #     return "The Employee might ATTRITION" if is_attrition_true > is_attrition_false else "Employee should STAY"

In [4]:
with gr.Blocks(title="Employee Attrition Prediction") as demo:
    gr.Markdown("""
                # 📊 Employee Attrition Prediction
                ## Dicoding - Solving HR Problem
                ### Made by : Muhammad Hafizh Dzaki
                ### Github Repo : [Here](https://github.com/hfzdzakii/Dicoding-SolvingHRProblem)
                """)
    with gr.Row():
        with gr.Column():
            gr.Markdown("""Compensation Matter Variables""")
            monthly_income = gr.Number(label="Monthly Income", 
                                       value=0, minimum=0, precision=2)
            stock_option_level = gr.Dropdown(label="Stock Option Level", 
                                             choices=["None", "Low", "Medium", "High"], 
                                             value="None")
            compensation = gr.Checkbox(label="Compensation Consideration Included?", value=False)
        with gr.Column():
            gr.Markdown("""Company Matter Variables""")
            environment_satisfaction = gr.Dropdown(label="Environment Satisfaction",
                                            choices=["Low", "Medium", "High", "Very High"], 
                                            value="Low")
            job_satisfaction = gr.Dropdown(label="Job Satisfaction",
                                            choices=["Low", "Medium", "High", "Very High"], 
                                            value="Low")
            over_time = gr.Radio(label="Working Over Time?",
                                choices=["Yes", "No"], value="No")
            relationship_satisfaction = gr.Dropdown(label="Relationship Satisfaction",
                                            choices=["Low", "Medium", "High", "Very High"], 
                                            value="Low")
            company = gr.Checkbox(label="Company Consideration Included?", value=False)
        with gr.Column():
            gr.Markdown("""Employee's Carrer Growth Matter Variables""")
            total_working_years = gr.Number(label="Total Working Years",
                                            value=0, minimum=0)
            years_in_current_role = gr.Number(label="Years in Current Role",
                                         value=0, minimum=0)
            years_since_last_promotion = gr.Number(label="Years Since Last Promotion",
                                                 value=0, minimum=0)
            career_growth = gr.Checkbox(label="Career Growth Consideration Included?", value=False)
    
    with gr.Row():
        with gr.Column():
            predict_btn = gr.Button("Predict", variant="primary")
            prediction = gr.Textbox(label="Prediction", interactive=False)
            
    predict_btn.click(
        fn=predict_attrition,
        inputs=[compensation, company, career_growth,
                monthly_income, stock_option_level,
                environment_satisfaction,
                job_satisfaction, over_time, relationship_satisfaction,
                total_working_years, years_in_current_role, 
                years_since_last_promotion
                ],
        outputs=prediction,
    )

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feat

Keyboard interruption in main thread... closing server.


In [ ]:
scaler.transform([[1,2,3,4,5,6,7,8,9,10,11,12,13,90]])

/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[-0.4047619 , -0.07158196,  0.07142857, -0.37142857, -0.05286993,
        -0.08383859,  0.77777778, -0.21428571,  0.225     ,  1.66666667,
         0.275     ,  0.66666667,  0.86666667,  5.29411765]])